In [ ]:
!pip install shapely
!pip install -i https://pypi.gurobi.com gurobipy

Looking in indexes: https://pypi.gurobi.com


In [ ]:
import numpy as np
from scipy.spatial import distance_matrix
from gurobipy import *
from scipy.spatial import ConvexHull
from shapely.geometry import Polygon, Point
from numpy import random

def generate_candidate_sites(points, M=100):
    # Convex Hull 알고리즘을 사용하여 지점들을 감싸는 다각형 생성
    hull = ConvexHull(points)
    polygon_points = points[hull.vertices]
    poly = Polygon(polygon_points)

    # 다각형의 경계 상자(Bounding Box)를 구하여 후보 서비스 지점 생성
    min_x, min_y, max_x, max_y = poly.bounds
    sites = []
    while len(sites) < M:
        random_point = Point([random.uniform(min_x, max_x),
                             random.uniform(min_y, max_y)])
        if (random_point.within(poly)):
            sites.append(random_point)

    return np.array([(p.x, p.y) for p in sites])

def mclp(points, K, radius, M):
    print("----- 설정 -----")
    print("  points 개수: %g" % points.shape[0])
    print("  K: %g" % K)
    print("  반경(Radius): %g" % radius)
    print("  M: %g" % M)

    import time
    start = time.time()

    # 후보 서비스 지점 생성
    sites = generate_candidate_sites(points, M)

    J = sites.shape[0]
    I = points.shape[0]
    D = distance_matrix(points, sites)

    # 거리가 반경 내에 있는 경우 1로, 아닌 경우 0으로 설정
    mask1 = D<=radius
    D[mask1] = 1
    D[~mask1] = 0

    # Gurobi 모델 생성
    m = Model()

    # 변수 추가
    x = {}
    y = {}
    for i in range(I):
      y[i] = m.addVar(vtype=GRB.BINARY, name="y%d" % i)
    for j in range(J):
      x[j] = m.addVar(vtype=GRB.BINARY, name="x%d" % j)
    m.update()

    # 제약조건 추가
    m.addConstr(quicksum(x[j] for j in range(J)) == K)

    for i in range(I):
        m.addConstr(quicksum(x[j] for j in np.where(D[i]==1)[0]) >= y[i])

    # 목적 함수 설정
    m.setObjective(quicksum(y[i]for i in range(I)), GRB.MAXIMIZE)

    # 출력 비활성화
    m.setParam("OutputFlag", 0)

    # 최적화 실행
    m.optimize()
    end = time.time()
    print("------- 결과 -------")
    print("  실행 시간: %s초" % float(end-start))
    print("  최적의 위치 후보 개수: %g" % m.objVal)

    solution = []
    if m.status == GRB.Status.OPTIMAL:
        for v in m.getVars():
            if v.x==1 and v.varName[0]=="x":
               solution.append(int(v.varName[1:]))
    opt_sites = sites[solution]
#     return opt_sites, m.objVal
    return opt_sites

In [ ]:
# 500개 3번 나눠서 뽑기
import numpy as np
import pandas as pd

# 데이터 로드
df = pd.read_csv("/Users/shy/Downloads/모델링_결과_StandardScaler.csv", encoding="CP949")
section1 = df[:601] # 167개
section2 = df[601:1202] # 167개
section3 = df[1202:1802] # 166개
print(len(section1), len(section2), len(section3))

601 601 600


In [ ]:
points = section2[["위도", "경도"]].values

# 선택할 위치의 개수
K = 166

# 각 위치의 반경
radius = 0.3

# 후보 위치 개수
M = 600

# mclp 실행
opt_sites = mclp(points, K, radius, M)

print("----------------------------------------", "\nMCLP 완료")

----- 설정 -----
  points 개수: 601
  K: 166
  반경(Radius): 0.3
  M: 600
------- 결과 -------
  실행 시간: 0.09453892707824707초
  최적의 위치 후보 개수: 601
---------------------------------------- 
MCLP 완료


In [ ]:
import pandas as pd

# 위도, 경도 분리
lats = opt_sites[:, 0]
lons = opt_sites[:, 1]

lats_list = []
lons_list = []

for lat in lats:
    lats_list.append(lat)

for lon in lons:
    lons_list.append(lon)

lats_lons = {
    "위도": lats_list,
    "경도": lons_list
}

result = pd.DataFrame(lats_lons)
result.to_csv("MCLP_결과.csv", encoding="CP949", index=False)

In [ ]:
result

,위도,경도
0,37.793765,127.259284
1,37.503938,127.402492
2,37.333175,127.512361
3,37.601933,127.293984
4,37.233245,127.203664
...,...,...
161,37.328264,126.946469
162,37.486077,127.559559
163,37.346033,127.306640
164,37.162049,127.094472


In [ ]:
result1 = pd.read_csv("/Users/shy/Downloads/MCLP_결과_1.csv", encoding="CP949")
result2 = pd.read_csv("/Users/shy/Downloads/MCLP_결과_2.csv", encoding="CP949")
result3 = pd.read_csv("/Users/shy/Downloads/MCLP_결과_3.csv", encoding="CP949")

result_df = pd.concat([result1, result2, result3], ignore_index=True)
result_df.to_csv("MCLP_결과.csv", encoding="CP949", index=False)

In [ ]:
result_df

,위도,경도
0,37.480659,126.760729
1,37.660576,127.372586
2,37.688398,127.014228
3,37.762219,126.915996
4,37.385502,127.184215
...,...,...
495,37.455701,127.518120
496,37.155598,127.147965
497,37.406533,127.607538
498,37.659357,126.830023
